This example shows how to perform sentiment analysis on YouTube comments using AgentQL with the [Google Colaboratory](https://colab.research.google.com/) environment.

Install [AgentQL](https://pypi.org/project/agentql/) library

In [ ]:
%pip install agentql

Install the Playwright dependency required by AgentQL.

In [ ]:
!playwright install chromium

You can [store](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) keys in Google Colab's secrets.

In [ ]:
import os

from google.colab import userdata

os.environ["AGENTQL_API_KEY"]=userdata.get('AGENTQL_API_KEY')
os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')

Run AgentQL script. Unfortunately, you can't see the browser window directly in Google Colab like you would on your local machine. However, let's still interact with it and take a screen recording of the browser session to see what’s happening. 

Please note that an online environment like Google Colab supports **asynchronous version** of AgentQL.

In [ ]:
import agentql
from openai import OpenAI
from playwright.async_api import async_playwright
from IPython.display import HTML
from base64 import b64encode

URL = "https://www.youtube.com/watch?v=JfM1mr2bCuk"

# Define the queries to interact with the page
QUERY = """
{
    video_title
    video_channel
    comments[] {
        comment_text
        author
    }
}
"""


async def get_comments():
    async with async_playwright() as playwright, await playwright.chromium.launch() as browser:
        # Set up the video recording
        context = await browser.new_context(
          record_video_dir="videos/",
          record_video_size={"width": 1280, "height": 720}
        )

        page = await agentql.wrap_async(await context.new_page())

        await page.goto(URL)


        for _ in range(5):
            # Wait for the page to load (helps to load the comments on the video)
            await page.wait_for_page_ready_state()

            # Scroll down the page to load more comments
            await page.keyboard.press("PageDown")

        # Use query_data() method to fetch the comments from the page
        response = await page.query_data(QUERY)

        return response


async def perform_sentiment_analysis(comments):
    USER_MESSAGE = "These are the comments on the video. I am trying to understand the sentiment of the comments."

    for comment in comments["comments"]:
        USER_MESSAGE += comment["comment_text"]

    SYSTEM_MESSAGE = """You are an expert in understanding the social media analytics and analysis and specialize in analyzing sentiment of the comments.
    Please find the comments on the video as follows:

    """

    USER_MESSAGE += "Could you please provide a summary of the comments on the video. Additionaly, just give only 3 takeaways which would be important for me as the creator of the video."

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    completion = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": USER_MESSAGE},
        ],
    )

    return completion.choices[0].message.content


comments = await get_comments()
summary = await perform_sentiment_analysis(comments)
print(summary)

# Display the video
video_dir = os.path.abspath("videos")
video_files = [f for f in os.listdir(video_dir) if f.endswith('.webm')]
if video_files:
    video_path = os.path.join(video_dir, video_files[0])
    with open(video_path, 'rb') as f:
      video_bytes = f.read()

    video_b64 = b64encode(video_bytes).decode('utf-8')
    video_html = f"""
    <video width="800" controls>
      <source src="data:video/webm;base64,{video_b64}" type="video/webm">
      Your browser does not support the video tag.
    </video>
    """
    display(HTML(video_html))
else:
    print("No video file was created")